# MNIST
Tanay Trivedi

In [0]:
import tensorflow as tf
import numpy as np

## Data

In [13]:
mnist = tf.contrib.learn.datasets.load_dataset("mnist")

x_train = mnist.train.images # Returns np.array
y_train = np.asarray(mnist.train.labels, dtype=np.int32)
x_test = mnist.test.images # Returns np.array
y_test = np.asarray(mnist.test.labels, dtype=np.int32)

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz


Create validation set...

In [0]:
valSetSize = .1 
valIdx = int((1-valSetSize)*len(x_train))

x_val = x_train[valIdx:]
y_val = y_train[valIdx:]
x_train = x_train[:valIdx]
y_train = y_train[:valIdx]

## Base Configuration

In [0]:
def cnn_model_fn(features, labels, mode): 
  #base configuration courtesy of  https://www.tensorflow.org/tutorials/
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=8,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=8,
      kernel_size=[4, 4],
      padding="same",
      activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
    
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 8])
    dense = tf.layers.dense(inputs=pool2_flat, units=5, activation=tf.nn.relu)
    dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    logits = tf.layers.dense(inputs=dropout, units=10)

    predictions = {
      "classes": tf.argmax(input=logits, axis=1),
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [0]:
config=tf.estimator.RunConfig(log_step_count_steps=1000)

mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn,config=config)

In [7]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_train},
    y=y_train,
    batch_size=100,
    num_epochs=None,
    shuffle=True)
mnist_classifier.train(
    input_fn=train_input_fn,
    steps=20000);

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpFyrjw0/model.ckpt.
INFO:tensorflow:loss = 2.2942145, step = 0
INFO:tensorflow:global_step/sec: 175.717
INFO:tensorflow:loss = 2.239706, step = 1000 (5.697 sec)
INFO:tensorflow:global_step/sec: 186.885
INFO:tensorflow:loss = 2.195702, step = 2000 (5.348 sec)
INFO:tensorflow:global_step/sec: 182.431
INFO:tensorflow:loss = 2.0860047, step = 3000 (5.484 sec)
INFO:tensorflow:global_step/sec: 184.07
INFO:tensorflow:loss = 2.1139343, step = 4000 (5.431 sec)
INFO:

### Validation

In [8]:
val_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_val},
    y=y_val,
    num_epochs=1,
    shuffle=False)
val_results = mnist_classifier.evaluate(input_fn=val_input_fn)
print(val_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-26-16:29:49
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpFyrjw0/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-09-26-16:29:50
INFO:tensorflow:Saving dict for global step 20000: accuracy = 0.8401818, global_step = 20000, loss = 0.8459055
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 20000: /tmp/tmpFyrjw0/model.ckpt-20000
{'loss': 0.8459055, 'global_step': 20000, 'accuracy': 0.8401818}


## Increase Dense Layer Size

In [0]:
def cnn_model_fn_1(features, labels, mode):
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=8,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=8,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 8])
    dense = tf.layers.dense(inputs=pool2_flat, units=10, activation=tf.nn.relu)
    dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    logits = tf.layers.dense(inputs=dropout, units=10)

    predictions = {
      "classes": tf.argmax(input=logits, axis=1),
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [6]:
config1=tf.estimator.RunConfig(log_step_count_steps=1000)

mnist_classifier1 = tf.estimator.Estimator(
    model_fn=cnn_model_fn_1,config=config1)

train_input_fn_1 = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_train},
    y=y_train,
    batch_size=100,
    num_epochs=None,
    shuffle=True)
mnist_classifier1.train(
    input_fn=train_input_fn_1,
    steps=20000);

INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0dfe9a3c10>, '_model_dir': '/tmp/tmpFfn7NH', '_protocol': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 1000, '_evaluation_master': '', '_eval_distribute': None, '_train_distribute': None, '_master': ''}
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflo

### Validation

In [7]:
val_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_val},
    y=y_val,
    num_epochs=1,
    shuffle=False)
val_results = mnist_classifier1.evaluate(input_fn=val_input_fn)
print(val_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-26-16:40:00
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpFfn7NH/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-09-26-16:40:00
INFO:tensorflow:Saving dict for global step 20000: accuracy = 0.83454543, global_step = 20000, loss = 0.77351105
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 20000: /tmp/tmpFfn7NH/model.ckpt-20000
{'loss': 0.77351105, 'global_step': 20000, 'accuracy': 0.83454543}


## Increase Dense Layer Again

In [0]:
def cnn_model_fn_2(features, labels, mode):
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=8,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=8,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 8])
    dense = tf.layers.dense(inputs=pool2_flat, units=1000, activation=tf.nn.relu)
    dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    logits = tf.layers.dense(inputs=dropout, units=10)

    predictions = {
      "classes": tf.argmax(input=logits, axis=1),
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [9]:
config2=tf.estimator.RunConfig(log_step_count_steps=1000)

mnist_classifier2 = tf.estimator.Estimator(
    model_fn=cnn_model_fn_2,config=config2)

train_input_fn_2 = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_train},
    y=y_train,
    batch_size=100,
    num_epochs=None,
    shuffle=True)
mnist_classifier2.train(
    input_fn=train_input_fn_2,
    steps=20000);

INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0dfa367f90>, '_model_dir': '/tmp/tmpReGpgK', '_protocol': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 1000, '_evaluation_master': '', '_eval_distribute': None, '_train_distribute': None, '_master': ''}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running lo

### Validation

In [10]:
val_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_val},
    y=y_val,
    num_epochs=1,
    shuffle=False)
val_results = mnist_classifier2.evaluate(input_fn=val_input_fn)
print(val_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-26-16:44:42
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpReGpgK/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-09-26-16:44:42
INFO:tensorflow:Saving dict for global step 20000: accuracy = 0.9681818, global_step = 20000, loss = 0.11163897
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 20000: /tmp/tmpReGpgK/model.ckpt-20000
{'loss': 0.11163897, 'global_step': 20000, 'accuracy': 0.9681818}


## Test

In [12]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x = {"x": x_test},
    y=y_test,
    num_epochs=1,
    shuffle=False)
test_results = mnist_classifier2.evaluate(input_fn=test_input_fn)
print(test_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-26-16:45:04
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpReGpgK/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-09-26-16:45:04
INFO:tensorflow:Saving dict for global step 20000: accuracy = 0.9622, global_step = 20000, loss = 0.12593603
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 20000: /tmp/tmpReGpgK/model.ckpt-20000
{'loss': 0.12593603, 'global_step': 20000, 'accuracy': 0.9622}
